In [1]:
# -*- coding: utf-8 -*-
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import os
import pickle
import re
import itertools

c:\Users\G-10\anaconda3\envs\tensor\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
##      뉴스기사
article = """
이르면 내년부터 국내 항공기 연료의 일정 비율 이상을 폐식용유와 팜유, 바이오디젤 등 ‘지속가능항공유’(SAF)로 채우는 정책이 시행된다. 유럽연합(EU)이 내년부터 EU 회원국 공항에서 이륙하는 모든 항공기에 SAF를 2% 이상 넣도록 의무화하는 등 ‘항공연료 탈탄소 정책’을 본격 시행하는 데 따른 것이다.

SAF 생산비가 일반 항공유(등유)의 2~5배에 달하는 만큼 향후 비행기 티켓값 상승으로 이어질 수 있다는 전망이 나온다.

19일 정유·항공업계에 따르면 정부는 오는 30일 이 같은 내용을 담은 ‘SAF 확대 전략’을 발표한다.

대책의 핵심은 항공유에 비해 탄소를 80% 적게 배출하는 SAF 사용을 의무화하는 것이다. 업계에서는 EU와 비슷하게 2% 안팎으로 시작한 뒤 단계별로 의무 사용량을 높일 것으로 예상하고 있다. EU는 2050년까지 SAF 사용률을 70%까지 끌어올리도록 못 박았다. 정부는 또 국내 공항에 SAF 급유시설을 설치하는 정유사에 예산을 지원하고 관련 규제를 풀어줄 계획이다. 조(兆) 단위 투자가 들어가는 SAF 전용시설 구축비에 대한 세액공제를 확대하는 방안도 추후 검토하기로 했다.

대한항공은 2022년부터 시작한 서울~파리 노선에 이어 30일 서울~도쿄 구간에 SAF를 채운 항공기를 띄운 뒤 순차적으로 확대할 계획이다.
SAF 시장 놓치면 '항공유 수출 1위' 흔들
SAF 탄소배출량 '석유 5분의1'…유럽은 내년부터 사용 의무화
한국은 세계에서 항공유를 가장 많이 수출하는 나라다. 중동에서 들여온 원유를 싸게 잘 정제하기 때문이다. 2022년 수출액(14조8270억원)은 휴대폰(10조2860억원)보다 많았다. 하지만 ‘미래의 항공유’로 불리는 지속가능항공유(SAF)를 놓고 보면 젬병이다. 미국 유럽 중국 등 전 세계에 323개나 있는 SAF 생산시설이 한국엔 하나도 없다.

정부가 오는 30일 ‘SAF 확산 전략’을 내놓는 이유가 여기에 있다. 당장 비용이 많이 든다고 SAF를 외면하면 미래 항공유 시장을 해외 정유업체에 통째로 내줄 수 있다는 것이다.
“SAF는 가야 할 길”
19일 국제항공운송협회(IATA)에 따르면 2050년 글로벌 SAF 수요는 4000억t을 넘어설 것으로 전망된다. 현재 연간 항공유 수요(3500억~4000억t)와 비슷한 수준이다. 탄소 배출 규제에 따라 미래에는 항공유 대부분이 SAF로 대체된다는 의미다. 폐식용유와 팜유, 바이오디젤, 폐목재 등을 사용하는 SAF는 일반 항공유에 비해 탄소를 80%가량 적게 배출한다.

이런 장점 덕분에 유럽과 미국을 중심으로 SAF 의무 사용을 요구하는 정책이 속속 등장하고 있다. 세계 최대 항공유 수입국인 미국은 2050년까지 일반 항공유를 100% SAF로 대체하기로 했다. 유럽은 내년 2%를 시작으로 2050년까지 모든 항공유의 70% 이상을 SAF로 사용하도록 할 예정이다.

글로벌 정유업계도 발 빠른 대응에 나섰다. 미국은 인플레이션 감축법(IRA)을 통해 L당 440~615원의 생산 보조금을 지급하고 있다. 일본도 기업의 SAF 생산설비 투자에 ‘그린이노베이션 기금’을 조성해 보조금을 지급하고 있다. 10년간 L당 270원의 세액공제 혜택도 준다. 독일과 네덜란드 역시 유럽연합(EU) 규정에 발맞춰 10년 이상의 장기 보조금 제도를 운용하고 있다.
한국은 정부 지원 없어
한국은 아무런 지원이 없다. 정유사에 주는 설비투자 보조금이나 세액공제는 물론 항공사에도 별다른 인센티브를 안 준다. 업계에선 SAF 전용 생산시설을 세우려면 정부가 도와줘야 한다고 주장한다. 50만t의 원료를 처리하는 전용 설비 하나를 만들려면 약 1조원의 투자금이 필요하기 때문이다. 생산 비용도 만만찮다. 등유보다 2~5배 더 든다.

정부는 일단 인천국제공항 등 국내 공항에 SAF 급유 시설을 설치하는 등 인프라부터 구축한다는 계획이다. 이에 필요한 예산을 지원하고 규제도 풀어주기로 했다. 정유사들은 SAF 생산시설 구축에 나선다. SK에너지는 기존 항공유 생산시설을 변경하는 시설 변경 투자를 통해 이른 시일 내에 SAF 상업생산에 들어간다는 계획이다. 에쓰오일, HD현대오일뱅크, GS칼텍스는 이와 별도로 SAF 전용 생산시설 설립도 추진하고 있다.

수요 창출은 항공사가 맡는다. 대한항공은 일본 도쿄 등 근거리 노선부터 SAF를 도입하기로 했다. SAF 사용 비율을 의무화하는 방안도 담긴다. 의무비율은 업계의 향후 실제 생산능력 등을 반영해 결정할 예정이다. 다만 업계가 요구해온 SAF 생산시설 투자세액공제(투자금의 15%)는 대책에 담기지 않아 ‘반쪽 지원’에 그쳤다는 평가도 나온다.
"""
article = article.replace('\n', '')
split_article = re.split(r'(\s*[a-zA-Z]+(?:[^\w\s]+[a-zA-Z]+)*\s*)', article)       #   한글과 영어 나누기
split_article = [part.strip() for part in split_article if part.strip()]            #   불필요한 공백 지우기

okt = Okt()                         #   형태소 분석기 Okt
nouns=[]
for sentence in split_article:
    sen = okt.nouns(sentence)
    if len(sen) != 0:               #   명사를 리스트에 추가
        nouns.extend(sen)
    else:
        nouns.append(sentence)      #   영어도 리스트에 추가(형태소 분석 후 길이가 0이면 영어로 판단)

text = ' '.join(nouns)              #   리스트를 하나의 문자열로

n_gram_range = (1,2)
word_vectorize = CountVectorizer(ngram_range=n_gram_range).fit([text])   #   1 ~ 2 글자로 N-Gram 생성
n_gram_words = word_vectorize.get_feature_names_out()

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')  #   Sentence Transformers
article_embedding = model.encode([article])
n_gram_embeddings = model.encode(n_gram_words)

def max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n, variety):
    #   뉴스 기사와 N-Gram의 유사도
    distances = cosine_similarity(article_embedding, n_gram_embeddings)
    #   N-Gram들 사이의 유사도
    distances_candidates = cosine_similarity(n_gram_embeddings, n_gram_embeddings)

    #   뉴스 기사와 유사도가 높은 N-Gram을 variety개 가져오기
    words_idx = list(distances.argsort()[0][-variety:])
    words_vals = [n_gram_words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    #   그 중 유사도가 가장 낮은 조합 찾기
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

news_keywords = max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n=5, variety=10)
print(f'뉴스 키워드 5개 : {news_keywords}')

c:\Users\G-10\anaconda3\envs\tensor\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


뉴스 키워드 5개 : ['탄소 정책', '글로벌 saf', '배출 규제', '정유 항공', '항공기 연료']


In [21]:
# 특정 디렉토리 경로
directory_path = 'D:/Topic_Modeling/New/embeddings'

# 모든 pkl 파일을 저장할 리스트
all_embeddings = []

# 파일 이름을 저장할 리스트
file_names = []

# 디렉토리 내의 모든 파일 확인
for filename in os.listdir(directory_path):
    if filename.endswith('.pkl'):
        file_path = os.path.join(directory_path, filename)
        
        # pkl 파일 불러오기
        with open(file_path, 'rb') as f:
            embedding = pickle.load(f)
            all_embeddings.append(embedding)

        # .pkl 확장자 제거한 파일 이름 저장
        name_without_extension = filename[:-4]  # 마지막 4글자(.pkl)를 제거
        file_names.append(name_without_extension)

# 결과 확인
print("임베딩 개수:", len(all_embeddings))
print("파일 이름 리스트:", len(file_names))

251
['가상화폐', '가스관', '가을', '개성공단', '건강기능식품', '건설', '건설기계장비부', '게임', '겨울', '고배당', '골판지', '공작기계', '광고', '괴질', '교육', '그래핀', '그린뉴딜', '금', '꼬마우선주', '남북경협', '네옴시티', '농업', '니켈', '닭고기', '당뇨', '대마', '대북주', '대체육', '독감', '돼지고기', '드론', '로봇', '롤러블폰', '리모델링', '리츠', '마스크', '마이크로바이옴', '말라리아', '메타버스', '면세점', '면역항암제', '무인화', '물류', '미세먼지', '미용기기', '미중무역전쟁', '밀가루', '바이오', '반도체', '반려동물', '방사능', '방산주', '방역관련주', '방탄소년단', '백화점', '변압기', '보안', '보험', '봄', '블록체인', '빅데이터', '빈대', '사료', '사물인터넷', '삼성그룹', '생체인식', '석유가스', '석유화학', '설탕', '소매유통', '수산물', '수소차', '스마트카', '스마트팩토리', '스마트폰', '스마트홈', '스팩', '시멘트', '시장주도주', '신용평가사', '아웃도어', '안철수', '애플', '양자암호통신', '양자컴퓨터', '엔터테인먼트', '여행', '영상콘텐츠', '영화', '오세훈', '온실가스', '요소수', '우주항공', '원자력발전', '유리기판', '유아용품', '윤석열', '은행', '음식료업종', '의료기기', '이낙연', '이미지센서', '이버멕틴', '이재명', '인테리어', '일자리', '자동차', '자동차부품', '자원개발', '자율주행', '저유가', '전기차', '전선', '전자결제', '정보보안', '제약', '조류독감', '조선', '종합상사', '주류', '줄기세포', '중국국적주', '증권', '지주사', '지진', '창투사', '천연가스', '철강', '철도', '초전도체', '치매', '치아', '카지노', '클라우드', '키오스

In [ ]:
all_embeddings[1]
distances = cosine_similarity(article_embedding, n_gram_embeddings)